<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/Seyoung/KR_WordRank(%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [ ]:
!pip install sentence_transformers

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Mecab
from konlpy.tag import Okt
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_spell_check.csv')
data.head()

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...


# **KR-WordRank**

In [14]:
%cd /content/drive/MyDrive/아이펠톤

/content/drive/MyDrive/아이펠톤


In [15]:
!ls

 일반대화csv생성.ipynb		       '일반대화추가 성능확인.ipynb'
 daily.csv			        krwordrank
'data_TFIDF(Verb,띄어쓰기추가).ipynb'   T5-finetuning2.ipynb
 DKTCdata			        T5-finetuning3.ipynb
'idx_2650 - train.csv'		        textrank
 허깅페이스.ipynb


In [138]:
def keyword(class_name):
    genre = data[data['class']==class_name].reset_index(drop=True).label_sentence

    attack, damaged = [], []

    for e in genre:
        e = e.split('\n')
        for e_ in e:
            if e_[0] == '1':
                attack.append(e_[2:])
            else:
                damaged.append(e_[2:])

    from krwordrank.sentence import summarize_with_sentences

    texts = attack
    keywords_att, sents = summarize_with_sentences(texts, num_keywords=50, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    texts = damaged
    keywords_dam, sents = summarize_with_sentences(texts, num_keywords=50, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    print('{}의 가해자 :'.format(class_name),keywords_att.keys())
    print()
    print('{}의 피해자 :'.format(class_name),keywords_dam.keys())
    
    keywords_att = list(keywords_att.keys())
    keywords_dam = list(keywords_dam.keys())
    
    target_attack, target_damaged = [], []

    for i in range(50):
        if (keywords_att[i] not in keywords_dam) : target_attack.append(keywords_att[i])
        if len(target_attack) == n: break 

    for i in range(50):
        if (keywords_dam[i] not in keywords_att) : target_damaged.append(keywords_dam[i])
        if len(target_damaged) == n: break
    
    print()
    print('{}의 가해자(중복제거) :'.format(class_name),target_attack)
    print()
    print('{}의 피해자(중복제거) :'.format(class_name),target_damaged)   

In [139]:
keyword('갈취 대화')

갈취 대화의 가해자 : dict_keys(['내가', '그럼', '아니', '지금', '그냥', '이거', '그래', '빌려', '내놔', '빨리', '진짜', '돈이', '우리', '친구', '오늘', '나도', '제가', '니가', '여기', '없어', '나오면', '있는', '당장', '너무', '그러', '어떻게', '있어', '좋은', '가져', '이번', '거기', '그거', '그렇게', '보내', '10', '뒤져서', '나한테', '얼마', '사람', '어이', '당신', '내일', '한번', '알아', '그건', '엄마', '이렇게', '얼른', '근데', '맞고'])

갈취 대화의 피해자 : dict_keys(['제발', '아니', '진짜', '없어', '이거', '안돼', '무슨', '네?', '내가', '돈이', '정말', '아.', '제가', '지금', '그건', '어떻게', '미안', '여기', '그래', '그게', '나도', '알겠', '네.', '그러', '그럼', '죄송', '엄마', '그거', '저희', '이건', '이게', '저번에도', '이번', '그런', '없는', '빌려', '빨리', '돌려', '우리', '근데', '그냥', '한번만', '너무', '이제', '이러', '오늘', '돈을', '안되', '얼마', '조금만'])

갈취 대화의 가해자(중복제거) : ['내놔', '친구', '니가', '나오면', '있는', '당장', '있어', '좋은', '가져', '거기', '그렇게', '보내', '10', '뒤져서', '나한테']

갈취 대화의 피해자(중복제거) : ['제발', '안돼', '무슨', '네?', '정말', '아.', '미안', '그게', '알겠', '네.', '죄송', '저희', '이건', '이게', '저번에도']


In [141]:
keyword('직장 내 괴롭힘 대화')

직장 내 괴롭힘 대화의 가해자 : dict_keys(['내가', '아니', '회사', '지금', '그럼', '오늘', '그래', '자네', '그렇게', '우리', '그냥', '이렇게', '김대리', '무슨', '사람', '하는', '이거', '일을', '진짜', '어떻게', '이번', '요즘', '하고', '너무', '제대로', '내일', '그런', '말이', '이런', '그러', '다른', '없어', '니가', '다시', '빨리', '아주', '보고', '누가', '휴가', '너가', '일이', '생각', '근데', '같이', '뭐가', '하면', '하나', '못하', '어디', '그게'])

직장 내 괴롭힘 대화의 피해자 : dict_keys(['제가', '네?', '죄송', '아니', '부장님', '무슨', '너무', '네.', '정말', '다시', '지금', '아.', '과장님', '아닙니다', '말씀', '저도', '그건', '그래도', '일이', '오늘', '그게', '팀장님', '이건', '저는', '그런', '하겠습니다.', '열심히', '사장님', '대리님', '알겠', '빨리', '업무', '진짜', '같습니다.', '그럼', '있어서', '아닌', '내일', '다른', '어떻게', '이번', '하고', '아직', '앞으로', '그렇게', '바로', '혹시', '그냥', '회사', '그러'])

직장 내 괴롭힘 대화의 가해자(중복제거) : ['내가', '그래', '자네', '우리', '이렇게', '김대리', '사람', '하는', '이거', '일을', '요즘', '제대로', '말이', '이런', '없어']

직장 내 괴롭힘 대화의 피해자(중복제거) : ['제가', '네?', '죄송', '부장님', '네.', '정말', '아.', '과장님', '아닙니다', '말씀', '저도', '그건', '그래도', '팀장님', '이건']


In [142]:
keyword('기타 괴롭힘 대화')

기타 괴롭힘 대화의 가해자 : dict_keys(['아니', '내가', '진짜', '그냥', '니가', '그래', '그럼', '우리', '여기', '그렇게', '지금', '너무', '무슨', '너가', '어떻게', '이거', '빨리', '사람', '그러', '어디', '얼굴', '이렇게', '뭐가', '너는', '냄새', '이게', '그런', '엄마', '새끼', '장애인', '친구', '돼지', '하는', '오늘', '근데', '나는', '그거', '하고', '없어', '애들', '기분', '너도', '뭐야', '말이', '못생', '같이', '너네', '저기', '다른', '이런'])

기타 괴롭힘 대화의 피해자 : dict_keys(['아니', '내가', '고객님', '무슨', '제발', '그만', '손님', '제가', '진짜', '지금', '너무', '죄송', '그런', '네?', '그렇게', '우리', '그게', '하지', '그래', '미안', '그냥', '말이', '나도', '나한테', '그러', '정말', '네.', '어떻게', '저희', '아.', '이거', '그건', '응?', '여기', '하는', '그럼', '자꾸', '사람', '너네', '있어', '오늘', '너가', '엄마', '니가', '혹시', '왜그', '선생님', '나는', '기분', '없어'])

기타 괴롭힘 대화의 가해자(중복제거) : ['빨리', '어디', '얼굴', '이렇게', '뭐가', '너는', '냄새', '이게', '새끼', '장애인', '친구', '돼지', '근데', '그거', '하고']

기타 괴롭힘 대화의 피해자(중복제거) : ['고객님', '제발', '그만', '손님', '제가', '죄송', '네?', '그게', '하지', '미안', '나도', '나한테', '정말', '네.', '저희']


In [145]:
keyword('협박 대화')

협박 대화의 가해자 : dict_keys(['내가', '지금', '니가', '진짜', '그럼', '그냥', '아니', '그래', '우리', '죽여', '너가', '당장', '어떻게', '사람', '빨리', '너도', '있어', '하나', '죽어', '나도', '그렇게', '당신', '죽고', '이렇게', '그러', '여기', '없어', '어디', '알아', '이게', '전에', '나한테', '오늘', '같이', '자꾸', '하면', '이제', '아주', '정신', '가족', '무슨', '이거', '말이', '있는', '생각', '칼로', '네가', '한번', '너네', '않으면'])

협박 대화의 피해자 : dict_keys(['제발', '아니', '내가', '제가', '정말', '무슨', '지금', '진짜', '우리', '죄송', '살려', '어떻게', '그래', '그러', '그게', '잘못', '경찰에', '너무', '그렇게', '이제', '그건', '그냥', '그만', '그런', '네?', '이거', '사람', '미안', '이러', '알겠', '건들지', '한번만', '신고', '나도', '선생님', '조금만', '시간을', '저는', '나는', '없어', '협박', '당장', '그럼', '니가', '너가', '당신', '네.', '다시', '빨리', '주세요'])

협박 대화의 가해자(중복제거) : ['죽여', '너도', '있어', '하나', '죽어', '죽고', '이렇게', '여기', '어디', '알아', '이게', '전에', '나한테', '오늘', '같이']

협박 대화의 피해자(중복제거) : ['제발', '제가', '정말', '죄송', '살려', '그게', '잘못', '경찰에', '너무', '그건', '그만', '그런', '네?', '미안', '이러']
